In [ ]:
## Task 1.1
import csv, datetime
csv_cursor = csv.reader(open("SUNSHINE.csv"))
next(csv_cursor)
data = list(
    filter(lambda x: x[0] < datetime.datetime(year=2023, month=1, day=1),
    map(lambda x: (datetime.datetime.strptime(x[0],"%Y-%m"),float(x[1])),csv_cursor)
    )
    )

def insert_sort(data):
    for i in range(1, len(data)):
        j = i
        while j> 0 and data[j][0] < data[j-1][0]:
            data[j], data[j-1] = data[j-1], data[j]
            j-= 1
        


In [ ]:
## for verification
insert_sort(data)
'''
f=open("sorted.csv","w")
for record in data:
    f.write(f"{record[0].year},{record[0].month},{record[1]}\n")
f.close()
'''
pass

In [ ]:
from datetime import datetime
## Task 1.2 group by year
def mean_by_year(data):
    cur_year = data[0][0].year
    lowest = data[0][1]
    highest = data[0][1]

    for _date , _mean in data[1:]:
        if _date.year  != cur_year: # change grouping
            #print statistics for previous grouping
            print(f"{cur_year} : Lowest:{lowest} Highest{highest} Range:{highest-lowest:.2f}" )

            cur_year = _date.year
            lowest = _mean
            highest = _mean
        else: # same grouping
            if _mean < lowest:
                lowest = _mean
            if _mean > highest:
                highest = _mean
    else:
        # print last group
        print(f"{cur_year} : Lowest:{lowest} Highest: {highest} Range:{highest-lowest:.2f}" )

In [ ]:
mean_by_year(data)

In [ ]:
## Task 1.3 group by months 1, to 12
from datetime import datetime
def mean_by_months(data):
    months = {} # {month:[]} ## all the means for month 1
    for _date , _mean in data:
        if _date.month in months:
            months[_date.month].append(_mean)
        else:
            months[_date.month] = []
    for month in months.keys():
        print(f"Month {month}: average daily sunshine duration 1982-2022 was { sum(months[month])/len(months[month]):.2f} hours")

    

In [ ]:
mean_by_months(data)

___ 
##### Using SQLite3

In [ ]:
import sqlite3, csv
mem = sqlite3.connect(":memory:")
mem.execute(''' 
CREATE TABLE IF NOT EXISTS stats  ( 
    Year INTEGER,
    Month INTEGER,
    Value REAL
)
''')
csv_cursor = csv.reader(open("SUNSHINE.csv"))
next(csv_cursor)
data = list(
    filter(lambda x: x[0] < 2023,
    map(lambda x: (int(x[0][:4]),int(x[0][5:]),float(x[1])),csv_cursor)
    )
    )
for row in data:
    mem.execute(''' 
    INSERT INTO stats VALUES (?,?,?)
    ''', row)
mem.commit()


In [ ]:
## Task 1.2
cursor = mem.execute('''
SELECT Year, MIN(Value) Lowest , MAX(Value) Highest, MAX(Value)-MIN(VAlue) Range
FROM stats
GROUP BY Year
ORDER By Year
''')
for result in cursor:
    print(f"{result[0]} : Lowest:{result[1]} Highest: {result[2]} Range:{result[3]:.2f}" )
    

In [29]:
# Task 1.3
cursor = mem.execute('''
SELECT Month, AVG(Value)
FROM stats
GROUP BY Month
''')
for result in cursor:
    print(f"Month {result[0]}: average daily sunshine duration 1982-2022 was { result[1]:.2f} hours")

Month 1: average daily sunshine duration 1982-2022 was 5.72 hours
Month 2: average daily sunshine duration 1982-2022 was 7.10 hours
Month 3: average daily sunshine duration 1982-2022 was 6.30 hours
Month 4: average daily sunshine duration 1982-2022 was 6.03 hours
Month 5: average daily sunshine duration 1982-2022 was 5.96 hours
Month 6: average daily sunshine duration 1982-2022 was 5.93 hours
Month 7: average daily sunshine duration 1982-2022 was 6.10 hours
Month 8: average daily sunshine duration 1982-2022 was 5.85 hours
Month 9: average daily sunshine duration 1982-2022 was 5.27 hours
Month 10: average daily sunshine duration 1982-2022 was 5.02 hours
Month 11: average daily sunshine duration 1982-2022 was 4.44 hours
Month 12: average daily sunshine duration 1982-2022 was 4.34 hours


In [30]:
mem.close()